In [1]:
import zipfile
from matplotlib import pyplot as plt
from collections import Counter
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
!unzip wiki-news-300d-1M.vec.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
!unzip crawl-300d-2M.vec.zip

^C
unzip:  cannot find or open crawl-300d-2M.vec.zip, crawl-300d-2M.vec.zip.zip or crawl-300d-2M.vec.zip.ZIP.


In [5]:
!wget -O Sherlock.txt https://www.gutenberg.org/ebooks/48320.txt.utf-8

--2023-10-26 02:56:45--  https://www.gutenberg.org/ebooks/48320.txt.utf-8
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://www.gutenberg.org/cache/epub/48320/pg48320.txt [following]
--2023-10-26 02:56:45--  http://www.gutenberg.org/cache/epub/48320/pg48320.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/cache/epub/48320/pg48320.txt [following]
--2023-10-26 02:56:45--  https://www.gutenberg.org/cache/epub/48320/pg48320.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622314 (608K) [text/plain]
Saving to: ‘Sherlock.txt’

Sherlock.txt        100%[===================>] 607.73

In [2]:
with open("Sherlock.txt", 'r', encoding='utf-8') as file:
    book = file.read()

In [3]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|no|No)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'

def split_into_sentences(text: str) -> list[str]:
    """
    Split the text into sentences.

    If the text contains substrings "<prd>" or "<stop>", they would lead
    to incorrect splitting because they are used as markers for splitting.

    :param text: text to be split into sentences
    :type text: str

    :return: list of sentences
    :rtype: list[str]
    """
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = ['<SOS> ' + s.strip() + ' <EOS>' for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences

In [4]:
book = split_into_sentences(book)

In [8]:
book[1000:1002]

['<SOS> I am going through the City first, and we can have some lunch on the way. <EOS>',
 '<SOS> I observe that there is a good deal of German music on the programme, which is rather more to my taste than Italian or French. <EOS>']

In [5]:
maxTokens = 5000
token = Tokenizer(num_words=maxTokens,
                  filters='!"“”#$%&()*+,.-/:;=?@[\\]^_`{|}~\t\n\ufeff\u2002', lower=True,
                  split=' ', char_level=False, oov_token="")
token.fit_on_texts(book)

In [6]:
sequences = token.texts_to_sequences(book)

In [51]:
textSequences = token.sequences_to_texts(sequences)

In [7]:
maxLen = len(max(sequences, key=len))

In [8]:
def create_dataset(sequence):
    inputs = []
    outputs = []
    for i, sent in enumerate(sequence):
        for j in range(1, len(sent)-1):
          inputs.append(sent[:j])
          outputs.append([sent[j]])

    return inputs, outputs

In [9]:
X, y = create_dataset(sequences)

In [22]:
token.sequences_to_texts([X[20]])

['<sos> the project gutenberg ebook of the adventures of sherlock holmes this ebook is for the use of anyone anywhere in']

In [23]:
maxLen = len(max(X, key=len))
x_train = pad_sequences(X, maxlen=maxLen)
reverse_dictionary = token.index_word
dictionary = dict([(value, key) for (key, value) in reverse_dictionary.items()])
num_words=len(dictionary)+1
y_train = np.array(tf.keras.utils.to_categorical(y, num_classes=num_words))

In [25]:
import codecs
import IPython.display as ipd
from IPython.display import Audio, update_display
from ipywidgets import IntProgress

#EMB = "crawl-300d-2M.vec"
EMB = "wiki-news-300d-1M.vec"
N = 1000000

#load embeddings
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open(f'./{EMB}', encoding='utf-8')
bar = IntProgress(min=0, max=N)
ipd.display(bar)

i = 0
for n, line in enumerate(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    if n//(N//100) > i:
          bar.value = n
          i += 1
f.close()
print('found %s word vectors' % len(embeddings_index))

loading word embeddings...


IntProgress(value=0, max=1000000)

found 999995 word vectors


In [26]:
embed_dim=300
embedding_matrix=np.zeros([num_words, embed_dim])
for word, idx in dictionary.items():
  if word in embeddings_index:
    embedding_matrix[idx,:]=embeddings_index[word]

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.3)

In [19]:
import pickle

FILE = './var.pick'
with open(FILE, 'wb') as file:
    # Serialize and write the variable to the file
    pickle.dump([embedding_matrix, x_train, x_test, y_train, y_test], file)

# Embedding Checkpoint

In [28]:
import pickle

FILE = './var.pick'
with open(FILE, 'rb') as file:
    # Deserialize and retrieve the variable from the file
    embedding_matrix, x_train, x_test, y_train, y_test = pickle.load(file)

In [29]:
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense, Input, Concatenate, Dot, RepeatVector, TimeDistributed, Multiply, Lambda, Flatten, Activation, Reshape, BatchNormalization
import tensorflow.keras.backend as K
from tensorflow.keras.activations import softmax
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

def softMaxOverTime(x):
    return softmax(x, axis=0)

nb_words = len(embedding_matrix) #Vocabulary size
embed_dim = len(embedding_matrix[0]) #Vectorization dim
value_dim = 100
maxLen = len(x_train[0])

In [40]:
input_layer = Input(shape=(maxLen,)) #Cantidad maxima de la frase de entrada, estos son los timesteps para este caso
embedding_layer = Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=maxLen, trainable=True)(input_layer)

#Contextualizador BidirRNN
lstm_out = Bidirectional(LSTM(value_dim, return_sequences=True, activation="tanh"), merge_mode="concat")(embedding_layer)
lstm_out = BatchNormalization()(lstm_out)
lstm_out = Dropout(0.2)(lstm_out)
ulog_attention = Dense(1, activation="linear")(lstm_out)

#Attention
attention = Activation(softMaxOverTime)(ulog_attention)
repeated_attention = TimeDistributed(RepeatVector(value_dim*2))(attention)
repeated_attention = Reshape([maxLen, value_dim*2])(repeated_attention)

#Pesamos el embedding con el score
weighted_embeddings = Multiply()([repeated_attention, lstm_out])
embedding_sum = Lambda(lambda x: K.sum(x, axis=1))(weighted_embeddings)

#Salida
dense1 = Dense(100, activation='relu')(embedding_sum)
dense2 = Dense(nb_words, activation='softmax')(dense1)
model = Model(input_layer , dense2)

In [ ]:
model.summary()

In [30]:
callbackROP = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5)
callbackES = EarlyStopping(monitor='val_loss', patience=10)

In [43]:
model.compile(loss="categorical_crossentropy", optimizer='rmsprop', metrics=["accuracy"])

In [44]:
model.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), batch_size=256, epochs=50, callbacks=[callbackROP, callbackES])

Epoch 1/50
 11/304 [>.............................] - ETA: 7:44 - loss: 9.0080 - accuracy: 0.0373

KeyboardInterrupt: 

# Model Checkpoint

In [9]:
model.save("./model")

In [31]:
model = load_model("./model")

In [43]:
aver = token.sequences_to_texts([[2952]])
aver

['confirm']

In [32]:
question = token.texts_to_sequences(["<sos> who is"])

# Stochastic beam search

In [46]:
import heapq

def beam_search(kBeams, maxOutputLen, candidates):

  if maxOutputLen <= 0:
    return [[]]*kBeams, [1]*kBeams
  else:
    for curr in candidates:
      #print(curr)
      out = pad_sequences([curr], maxlen=maxLen)
      out = model.predict(out, verbose=0)
      newCandidates = np.argsort(out[0])[::-1][:kBeams] #Agarramos los indices de los k-elementos con mas prob
      newCandidates = [[num] for num in newCandidates] #Convertimos los numeros en listas de numeros por compatibilidad con pad_sequences
    
    newCandProba = np.sort(out[0])[::-1][:kBeams] #Agarramos su probabilidad

      print(newCandProba)
      print(newCandidates)

    return np.concatenate((newCandidates, beam_search(kBeams, maxOutputLen-1, newCandidates)[0]), axis=1), np.multiply(newCandProba,  beam_search(kBeams, maxOutputLen-1, newCandidates)[1])

In [47]:
maxOutputLen = 2
kBeams = 4
candidates = question

outSeq, outProb = beam_search(kBeams, maxOutputLen, candidates)
result = token.sequences_to_texts(outSeq)

for i, sent in enumerate(result):
  print(f"prob: {outProb[i]:.2f} - Rta: {sent}")

[1.0000000e+00 2.1127763e-19 0.0000000e+00 0.0000000e+00]
[[4], [9], [8843], [2952]]
[0.9981997  0.00180031 0.         0.        ]
[[1], [41], [8843], [2945]]
[1.000000e+00 4.429616e-28 0.000000e+00 0.000000e+00]
[[1], [41], [8843], [2945]]
[1.0000000e+00 4.0630383e-24 0.0000000e+00 0.0000000e+00]
[[7], [6], [8843], [2946]]
[1.0000000e+00 2.6802279e-28 0.0000000e+00 0.0000000e+00]
[[6], [11], [8843], [2946]]
[0.9981997  0.00180031 0.         0.        ]
[[1], [41], [8843], [2945]]
[1.000000e+00 4.429616e-28 0.000000e+00 0.000000e+00]
[[1], [41], [8843], [2945]]
[1.0000000e+00 4.0630383e-24 0.0000000e+00 0.0000000e+00]
[[7], [6], [8843], [2946]]
[1.0000000e+00 2.6802279e-28 0.0000000e+00 0.0000000e+00]
[[6], [11], [8843], [2946]]
prob: 1.00 - Rta: the i
prob: 0.00 - Rta: a that
prob: 0.00 - Rta:  
prob: 0.00 - Rta: confirm blows


# Greedy search

In [47]:
response = []
reply = question
out = 0
finished = False

while out != token.texts_to_sequences(["<eos>"])[0][0] and not finished:
    out = pad_sequences(reply, maxlen=maxLen)
    out = model.predict(out, verbose=0)
    out = int(np.argmax(out))
    reply = [[out]]
    if out in response:
      finished = True
    else:
      response.append(out)

In [48]:
def from_seq_to_text(seq):
    return token.sequences_to_texts([seq])

In [49]:
rta = from_seq_to_text(response)
rta

['']